In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
import re
import numpy as np

import xgboost as xgb
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.metrics import accuracy_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from keras.models import Model, Sequential
from keras.layers import Input, Dense, Concatenate, Reshape, Dropout, BatchNormalization, Embedding, Flatten, LSTM

C:\Users\Maedr3\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 16

In [3]:
data = pd.read_csv('../../chapter 8/data/movie_reviews.csv', encoding='latin-1')

In [4]:
data.shape

(25000, 2)

In [5]:
data.SentimentText = data.SentimentText.str.lower()

In [6]:
def clean_str(string):
    
    string = re.sub(r"https?\://\S+", '', string)
    string = re.sub(r'\<a href', ' ', string)
    string = re.sub(r'&amp;', '', string) 
    string = re.sub(r'<br />', ' ', string)
    string = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', string)
    string = re.sub('\d','', string)
    string = re.sub(r"can\'t", "cannot", string)
    string = re.sub(r"it\'s", "it is", string)
    return string

In [7]:
data.SentimentText = data.SentimentText.apply(lambda x: clean_str(str(x)))

In [8]:
pd.Series(' '.join(data['SentimentText']).split()).value_counts().head(10)

movie    43558
film     39095
it       30659
one      26509
is       20355
like     20270
good     15099
the      13913
time     12682
even     12656
dtype: int64

In [9]:
stop_words = stopwords.words('english') + ['movie', 'film', 'time']
stop_words = set(stop_words)
remove_stop_words = lambda r: [[word for word in word_tokenize(sente) if word not in stop_words] for sente in sent_tokenize(r)]
data['SentimentText'] = data['SentimentText'].apply(remove_stop_words)

In [10]:
def combine_text(text):    
    try:
        return ' '.join(text[0])
    except:
        return np.nan

In [11]:
data.SentimentText = data.SentimentText.apply(lambda x: combine_text(x))

In [12]:
data = data.dropna(how='any')

In [13]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(list(data['SentimentText']))
sequences = tokenizer.texts_to_sequences(data['SentimentText'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 77348 unique tokens.


In [15]:
reviews = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [16]:
def load_embedding(filename, word_index , num_words, embedding_dim):
    embeddings_index = {}
    file = open(filename, encoding="utf-8")
    for line in file:
        values = line.split()
        word = values[0]
        coef = np.asarray(values[1:])
        embeddings_index[word] = coef
    file.close()
    
    embedding_matrix = np.zeros((num_words, embedding_dim))
    for word, pos in word_index.items():
        if pos >= num_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[pos] = embedding_vector
    return embedding_matrix

In [17]:
embedding_matrix = load_embedding('movie_embedding.txt', word_index, len(word_index), EMBEDDING_DIM)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(reviews, pd.get_dummies(data.Sentiment), test_size=0.2, random_state=9)

In [38]:
inp = Input((MAX_SEQUENCE_LENGTH,))
embedding_layer = Embedding(len(word_index),
                    EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=MAX_SEQUENCE_LENGTH,
                    trainable=False)(inp)
model = Dropout(0.10)(embedding_layer)
model = BatchNormalization()(model)
model = LSTM(256, dropout=0.1, return_sequences=True)(model)
model = LSTM(256, dropout=0.1)(model)
model = Dense(units=128, activation='relu')(model)
model = Dense(units=32, activation='relu')(model)
model = Dropout(0.3)(model)
predictions = Dense(units=2, activation='softmax')(model)
model = Model(inputs = inp, outputs = predictions)

model.compile(loss='binary_crossentropy', optimizer='sgd', metrics = ['acc'])

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 100, 16)           1237568   
_________________________________________________________________
dropout_14 (Dropout)         (None, 100, 16)           0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 100, 16)           64        
_________________________________________________________________
lstm_8 (LSTM)                (None, 100, 256)          279552    
_________________________________________________________________
lstm_9 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_14 (Dense)             (None, 128)               32896     
__________

In [40]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=10, batch_size=256)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 343s 17ms/step - loss: 0.6901 - acc: 0.5492 - val_loss: 0.6836 - val_acc: 0.6530
Epoch 2/10
20000/20000 [==============================] - 665s 33ms/step - loss: 0.6768 - acc: 0.6477 - val_loss: 0.6651 - val_acc: 0.6950
Epoch 3/10
20000/20000 [==============================] - 684s 34ms/step - loss: 0.6564 - acc: 0.6828 - val_loss: 0.6384 - val_acc: 0.7106
Epoch 4/10
20000/20000 [==============================] - 685s 34ms/step - loss: 0.6281 - acc: 0.7014 - val_loss: 0.6054 - val_acc: 0.7178
Epoch 5/10
20000/20000 [==============================] - 435s 22ms/step - loss: 0.5966 - acc: 0.7039 - val_loss: 0.5702 - val_acc: 0.7268
Epoch 6/10
20000/20000 [==============================] - 377s 19ms/step - loss: 0.5704 - acc: 0.7164 - val_loss: 0.5429 - val_acc: 0.7380
Epoch 7/10
20000/20000 [==============================] - 427s 21ms/step - loss: 0.5537 - acc: 0.7248 - val_loss: 0.5

In [41]:
preds = model.predict(X_test)

In [42]:
accuracy_score(np.argmax(preds, 1), np.argmax(y_test.values, 1))

0.7692

In [43]:
y_actual = pd.Series(np.argmax(y_test.values, axis=1), name='Actual')
y_pred = pd.Series(np.argmax(preds, axis=1), name='Predicted')
pd.crosstab(y_actual, y_pred, margins=True)

Predicted,0,1,All
Actual,,,
0,1922,531,2453
1,623,1924,2547
All,2545,2455,5000


In [46]:
review_num = 110
print("Review: \n"+tokenizer.sequences_to_texts([X_test[review_num]])[0])
sentiment = "Positive" if np.argmax(preds[review_num]) else "Negative"
print("\nPredicted sentiment = "+ sentiment)
sentiment = "Positive" if np.argmax(y_test.values[review_num]) else "Negative"
print("\nActual sentiment = "+ sentiment)

Review: 
warning spoilers really stupid group young italy find warrior souls one wears becomes possessed spirit demon killing several friends die blade demon corpse waste viewers fine young ladies leave clothes gore ludicrous best acting terrible perfect bad script

Predicted sentiment = Negative

Actual sentiment = Negative
